Works only with work/CMSSW_10_2_3/src

In [1]:
import os, re, json
import pickle
import commands
from glob import glob
import numpy as np
from prettytable import PrettyTable
import yaml

In [2]:
!export PATH=$HOME/.local/bin:/cvmfs/cms-bril.cern.ch/brilconda/bin:$PATH

# Input data

In [3]:
# loc = os.environ['HOME'] + '/work/CMSSW_10_2_3/src/ntuplizer/BPH_RDntuplizer/jobSubmission/tmp'
# loc += '/crab_ParkingBPH1_Run2018D-05May2019promptD-v1_RDntuplizer_B2DstMu_191126'
# print loc
# loc = glob(loc)
# print len(loc)
# tag = 'B2DstMu_191126'

In [4]:
# loc = os.environ['HOME'] + '/work/CMSSW_10_2_3/src/ntuplizer/BPH_RDntuplizer/jobSubmission/tmp'
# loc += '/crab_ParkingBPH*_RDntuplizer_B2JpsiKst_200124'
# print loc
# loc = glob(loc)
# print len(loc)

In [5]:
loc = os.environ['HOME'] + '/work/CMSSW_10_2_3/src/ntuplizer/BPH_RDntuplizer/jobSubmission/tmp'
loc += '/crab_ParkingBPH*_RDntuplizer_B2DstMu_200327'
print loc
loc = glob(loc)
print len(loc)

/storage/user/ocerri/work/CMSSW_10_2_3/src/ntuplizer/BPH_RDntuplizer/jobSubmission/tmp/crab_ParkingBPH*_RDntuplizer_B2DstMu_200327
5


In [6]:
nameTemplate = '../data/cmsRD/lumiReport/{}_brilcalcPerTrigger.yaml'
recreate = True

lumiInfo = {}
for main_dir in loc:
    print main_dir.split('/')[-1]
    s = main_dir.split('/')[-1]
    idx = s.find('Run2018')
    era = s[idx+len('Run2018')]
    idx = s.find('ParkingBPH')
    part = int(s[idx+len('ParkingBPH')])
    print era, part

    if not era in lumiInfo.keys():
        lumiInfo[era] = {}
        
    name = os.path.basename(main_dir)[5:]
    fname = nameTemplate.format(name)
    if os.path.isfile(fname) and not recreate:
        lumiDic = yaml.load(open(fname, 'rb'))
    else:
        cmd = 'brilcalc lumi -u /fb --precision 2f'
        cmd += ' --normtag /cvmfs/cms-bril.cern.ch/cms-lumi-pog/Normtags/normtag_PHYSICS.json'
        cmd += ' --hltpath HLT_Mu\*_IP?_part' + str(part-1) + '_v?'
        cmd += ' -i {}/processedLumis.json'.format(main_dir + '/results')
        if os.uname()[1] == 'login-1.hep.caltech.edu':
            cmd += ' -c web'
        
        status, output = commands.getstatusoutput(cmd)
        aux = output.split('#Summary:')[1]
        aux = aux.split('\n')
        lumiDic = {}
        for l in aux:
            if not l.startswith('| HLT'):
                continue
            l = l[1:-1]
            content = l.split(' | ')
            trgPath = content[0].replace(' ', '')
            lumi = float(content[-1].replace(' ', ''))
            lumiDic[trgPath] = lumi
            
        lumi_str = yaml.dump(lumiDic, default_flow_style=False)
        f = open(fname, 'w')
        f.write(lumi_str)
        f.close()
            
    for trgPath, lumi in lumiDic.iteritems():
        if not '_part'+str(part-1) in trgPath:
            continue
        trgPath = trgPath.replace('_part' + str(part-1), '')          
        if lumi < 0.02:
            continue
        if not trgPath in lumiInfo[era].keys():
            lumiInfo[era][trgPath] = np.zeros(6)
        lumiInfo[era][trgPath][part-1] = lumi
        print trgPath, part, lumi
    print '\n'

crab_ParkingBPH1_Run2018D-05May2019promptD-v1_RDntuplizer_B2DstMu_200327
D 1
HLT_Mu8_IP6_v2 1 1.65
HLT_Mu7_IP4_v2 1 1.27
HLT_Mu12_IP6_v2 1 5.25
HLT_Mu8_IP5_v2 1 1.65
HLT_Mu8_IP3_v3 1 0.08
HLT_Mu9_IP6_v3 1 4.12
HLT_Mu9_IP5_v2 1 3.33


crab_ParkingBPH5_Run2018D-05May2019promptD-v1_RDntuplizer_B2DstMu_200327
D 5
HLT_Mu12_IP6_v2 5 5.29
HLT_Mu8_IP5_v2 5 1.65
HLT_Mu8_IP3_v3 5 0.08
HLT_Mu9_IP6_v3 5 4.15
HLT_Mu8_IP6_v2 5 1.65
HLT_Mu9_IP5_v2 5 3.36
HLT_Mu7_IP4_v2 5 1.28


crab_ParkingBPH2_Run2018D-05May2019promptD-v1_RDntuplizer_B2DstMu_200327
D 2
HLT_Mu8_IP3_v3 2 0.08
HLT_Mu8_IP6_v2 2 1.59
HLT_Mu7_IP4_v2 2 1.24
HLT_Mu8_IP5_v2 2 1.59
HLT_Mu12_IP6_v2 2 5.13
HLT_Mu9_IP5_v2 2 3.26
HLT_Mu9_IP6_v3 2 4.02


crab_ParkingBPH3_Run2018D-05May2019promptD-v1_RDntuplizer_B2DstMu_200327
D 3
HLT_Mu9_IP5_v2 3 3.36
HLT_Mu7_IP4_v2 3 1.28
HLT_Mu8_IP3_v3 3 0.08
HLT_Mu12_IP6_v2 3 5.29
HLT_Mu8_IP5_v2 3 1.65
HLT_Mu9_IP6_v3 3 4.14
HLT_Mu8_IP6_v2 3 1.65


crab_ParkingBPH4_Run2018D-05May2019promptD-v1_RDntuplizer_B2DstM

In [7]:
lumiInfo

{'D': {'HLT_Mu12_IP6_v2': array([5.25, 5.13, 5.29, 5.29, 5.29, 0.  ]),
  'HLT_Mu7_IP4_v2': array([1.27, 1.24, 1.28, 1.28, 1.28, 0.  ]),
  'HLT_Mu8_IP3_v3': array([0.08, 0.08, 0.08, 0.08, 0.08, 0.  ]),
  'HLT_Mu8_IP5_v2': array([1.65, 1.59, 1.65, 1.64, 1.65, 0.  ]),
  'HLT_Mu8_IP6_v2': array([1.65, 1.59, 1.65, 1.64, 1.65, 0.  ]),
  'HLT_Mu9_IP5_v2': array([3.33, 3.26, 3.36, 3.35, 3.36, 0.  ]),
  'HLT_Mu9_IP6_v3': array([4.12, 4.02, 4.14, 4.14, 4.15, 0.  ])}}

In [8]:
eras = list(np.sort(lumiInfo.keys()))
table = PrettyTable()

table.field_names = ['HLT path'] + eras

printed_paths = []
for e in eras:
    for t in lumiInfo[e]:
        t = t[:-3]
        if t in printed_paths:
            continue
        printed_paths.append(t)
        row = [t]
        for e in eras:
            if t in [path[:-3] for path in lumiInfo[e].keys()]:
                for path in lumiInfo[e].keys():
                    if t == path[:-3]:
                        row.append(str(lumiInfo[e][path]))
                        break
            else:
                row.append(' - ')
        table.add_row(row)
print table

+--------------+---------------------------------+
|   HLT path   |                D                |
+--------------+---------------------------------+
| HLT_Mu8_IP3  | [0.08 0.08 0.08 0.08 0.08 0.  ] |
| HLT_Mu9_IP5  | [3.33 3.26 3.36 3.35 3.36 0.  ] |
| HLT_Mu8_IP6  | [1.65 1.59 1.65 1.64 1.65 0.  ] |
| HLT_Mu7_IP4  | [1.27 1.24 1.28 1.28 1.28 0.  ] |
| HLT_Mu12_IP6 | [5.25 5.13 5.29 5.29 5.29 0.  ] |
| HLT_Mu8_IP5  | [1.65 1.59 1.65 1.64 1.65 0.  ] |
| HLT_Mu9_IP6  | [4.12 4.02 4.14 4.14 4.15 0.  ] |
+--------------+---------------------------------+


In [9]:
eras = list(np.sort(lumiInfo.keys()))
table = PrettyTable()

l = ['pT', 'IP']
for e in eras:
    l += [e, 'Tot ' + e]
l += ['Tot']
table.field_names = l

printed_paths = []
for e in eras:
    for t in lumiInfo[e]:
        tot_lumi = 0
        t = t[:-3]
        if t in printed_paths:
            continue
        printed_paths.append(t)
        row = [t[6:-4], t[-1]]
        for e in eras:
            if t in [path[:-3] for path in lumiInfo[e].keys()]:
                for path in lumiInfo[e].keys():
                    if t == path[:-3]:
                        row.append(str(lumiInfo[e][path]))
                        
                        s = np.sum(lumiInfo[e][path])
                        tot_lumi += s
                        row.append('{:.2f}'.format(s))
                        break
            else:
                row.append(' - ')
                row.append(' - ')
        row.append('{:.2f}'.format(tot_lumi))
        table.add_row(row)
print table

+----+----+---------------------------------+-------+-------+
| pT | IP |                D                | Tot D |  Tot  |
+----+----+---------------------------------+-------+-------+
| 8  | 3  | [0.08 0.08 0.08 0.08 0.08 0.  ] |  0.40 |  0.40 |
| 9  | 5  | [3.33 3.26 3.36 3.35 3.36 0.  ] | 16.66 | 16.66 |
| 8  | 6  | [1.65 1.59 1.65 1.64 1.65 0.  ] |  8.18 |  8.18 |
| 7  | 4  | [1.27 1.24 1.28 1.28 1.28 0.  ] |  6.35 |  6.35 |
| 12 | 6  | [5.25 5.13 5.29 5.29 5.29 0.  ] | 26.25 | 26.25 |
| 8  | 5  | [1.65 1.59 1.65 1.64 1.65 0.  ] |  8.18 |  8.18 |
| 9  | 6  | [4.12 4.02 4.14 4.14 4.15 0.  ] | 20.57 | 20.57 |
+----+----+---------------------------------+-------+-------+
